In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

import os
import tensorflow as tf
import keras
import cv2

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.applications.vgg19 import VGG19
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPool2D,BatchNormalization, LSTM,MaxPooling2D
from keras.models import Sequential
from keras import regularizers
import keras

from keras.layers import TimeDistributed
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = '/kaggle/input/fer2013/train/'
test_dir = '/kaggle/input/fer2013/test/'
print(os.listdir(train_dir))
print(os.listdir(test_dir))

In [ ]:
def Classes_Count( path, name):
    Classes_Dict = {}
    
    for Class in os.listdir(path):
        
        Full_Path = path + Class
        Classes_Dict[Class] = len(os.listdir(Full_Path))
        
    df = pd.DataFrame(Classes_Dict, index=[name])
    
    return df

Train_Count = Classes_Count(train_dir, 'Train').transpose().sort_values(by="Train", ascending=False)
Test_Count = Classes_Count(test_dir, 'Test').transpose().sort_values(by="Test", ascending=False)

In [ ]:
pd.concat([Train_Count,Test_Count] , axis=1)

In [ ]:
plt.figure(figsize=(10, 6),dpi=150)
sns.barplot(x=Train_Count.index, y='Train', data=Train_Count)
plt.title('Train Values per Emotion Category')
plt.xlabel('Emotion')
plt.ylabel('Training image Count')

In [ ]:
plt.figure(figsize=(10, 6),dpi=150)
sns.barplot(x=Test_Count.index, y='Test', data=Test_Count)
plt.title('Test Values per Emotion Category')
plt.xlabel('Emotion')
plt.ylabel('Testing image Count')

In [ ]:
plt.style.use('default')
plt.figure(figsize = (25, 8))
image_count = 1
BASE_URL = '/kaggle/input/fer2013/train/'

for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(1, 7, image_count)
                image_count += 1
                image = cv2.imread(BASE_URL + directory + '/' + file)
                image_shape = image.shape
                print("Image Shape:", image_shape)
                plt.imshow(image)
                plt.title(directory, fontsize = 20)

In [ ]:
img_shape = 48
batch_size = 128
train_data_path = '/kaggle/input/fer2013/train'
test_data_path = '/kaggle/input/fer2013/test'

In [ ]:
train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        # Data Augmentation
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,                                        
        fill_mode='nearest',
    )


test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
)

In [ ]:
train_data = train_preprocessor.flow_from_directory(
    train_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode='grayscale', 
    shuffle=True,
    batch_size=batch_size,
    subset='training', 
)


test_data = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode='grayscale',
    shuffle=False,
    batch_size=batch_size,
)

In [ ]:
class SqueezeExciteBlock(tf.keras.layers.Layer):
    def __init__(self, units, ratio=16):
        super(SqueezeExciteBlock, self).__init__()
        self.units = units
        self.ratio = ratio

    def build(self, input_shape):
        num_channels = input_shape[-1]
        
        self.squeeze = tf.keras.layers.GlobalAveragePooling2D()
        self.excitation = tf.keras.Sequential([
            tf.keras.layers.Dense(num_channels // self.ratio, activation='relu'),
            tf.keras.layers.Dense(num_channels, activation='sigmoid'),
        ])
        self.reshape = tf.keras.layers.Reshape((1, 1, num_channels))

    def call(self, inputs):
        squeezed = self.squeeze(inputs)
        excited = self.excitation(squeezed)
        excited = self.reshape(excited)
        scaled_input = tf.keras.layers.Multiply()([inputs, excited])
        return scaled_input

In [ ]:
def Create_CNN_Model():
    model = Sequential()

    # CNN1
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_shape, img_shape, 1)))
    model.add(BatchNormalization())
    
    # Add Squeeze-and-Excitation after CNN1
    model.add(SqueezeExciteBlock(64))

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.25))

    # CNN2
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())

    # Add Squeeze-and-Excitation after CNN2
    model.add(SqueezeExciteBlock(128))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.25))

    # CNN3
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())

    # Add Squeeze-and-Excitation after CNN3
    model.add(SqueezeExciteBlock(256))

    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.25))
    
    model.add(SqueezeExciteBlock(256))

    # Global Average Pooling
    model.add(GlobalAveragePooling2D())

    # Fully Connected Layers
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(7, activation='softmax'))

    return model

In [ ]:
CNN_Model = Create_CNN_Model()

CNN_Model.summary()

CNN_Model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy']) #Lr=0.001

In [ ]:
# Callback Checkpoint
checkpoint_path = "CNN_Model_With_Squeeze_Checkpoint.tf"

Checkpoint = ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True,mode='max',verbose=1)

# Early Stopping Callback to monitor the accuracy
Early_Stopping = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True, verbose=1)

# ReduceLROnPlateau Callback to reduce overfitting by decreasing learning rate
Reducing_LR = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=0.000005,
    verbose=1
)


callbacks = [Early_Stopping, Reducing_LR]

steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = test_data.n // test_data.batch_size

In [ ]:
CNN_history = CNN_Model.fit( train_data , validation_data= test_data , epochs=50, batch_size= batch_size,
                            callbacks=callbacks, steps_per_epoch= steps_per_epoch, validation_steps=validation_steps)

In [ ]:
hist=CNN_history.history
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])
plt.title("Accuracy plot")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("CNNv2_accuracy.png")

In [ ]:
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.title("Accuracy loss")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig("CNNv2_loss.png")

In [ ]:
# Save the entire model (architecture + weights)
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
model_filename = f"/kaggle/working/CNN_Modelv3Sq_{timestamp}.tf"
model_weightsfilename = f"/kaggle/working/CNN_Weights_Modelv3Sq_{timestamp}.tf"

CNN_Model.save(model_filename)
CNN_Model.save_weights(model_weightsfilename)